# 第7章 文本数据
2020.06.24

In [213]:
import pandas as pd
import numpy as np

## 一、string类型的性质

### 1. string与object的区别
#### string类型和object不同之处有三：
#### ① 字符存取方法（string accessor methods，如str.count）会返回相应数据的Nullable类型，而object会随缺失值的存在而改变返回类型
#### ② 某些Series方法不能在string上使用，例如： Series.str.decode()，因为存储的是字符串而不是字节
#### ③ string类型在缺失值存储或运算时，类型会广播为pd.NA，而不是浮点型np.nan
#### 其余全部内容在当前版本下完全一致，但迎合Pandas的发展模式，我们仍然全部用string来操作字符串

### 2. string类型的转换
#### 如果将一个其他类型的容器直接转换string类型可能会出错：

In [217]:
#pd.Series([1,'1.']).astype('string') #报错 ValueError: StringArray requires a sequence of strings or pandas.NA
#pd.Series([1,2]).astype('string') #报错 ValueError: StringArray requires a sequence of strings or pandas.NA
#pd.Series([True,False]).astype('string') #报错 ValueError: StringArray requires a sequence of strings or pandas.NA

#### 当下正确的方法是分两部转换，先转为str型object，在转为string类型：

In [218]:
pd.Series([1,'1.']) # 原始类型是 int64 和 str 混合的 object
#pd.Series([1,'1.']).astype('string') #直接转换会报错 ValueError: StringArray requires a sequence of strings or pandas.NA

0     1
1    1.
dtype: object

In [219]:
pd.Series([1,'1.']).astype('str') # 第一步转为 str 的object

0     1
1    1.
dtype: object

In [220]:
pd.Series([1,'1.']).astype('str').astype('string') # 第二步转为 string 类型

0     1
1    1.
dtype: string

In [221]:
pd.Series([1,2]) #原始类型是 int64
#pd.Series([1,2]).astype('string') #直接转换会报错--ValueError: StringArray requires a sequence of strings or pandas.NA

0    1
1    2
dtype: int64

In [222]:
pd.Series([1,2]).astype('str') # 第一步转为 str 类型的 object

0    1
1    2
dtype: object

In [223]:
pd.Series([1,2]).astype('str').astype('string') # 第二步转为 string 类型

0    1
1    2
dtype: string

In [224]:
pd.Series([True,False]) # 原始类型是布尔型
#pd.Series([True,False]).astype('string') #直接转换会报错 ValueError: StringArray requires a sequence of strings or pandas.NA

0     True
1    False
dtype: bool

In [225]:
pd.Series([True,False]).astype('str') #第一步转为 str 类型的object

0     True
1    False
dtype: object

In [226]:
pd.Series([True,False]).astype('str').astype('string') #第二步转为 string 类型

0     True
1    False
dtype: string

* 但这种方法也不是特别正确, 因为会把缺失值也按字面显示转为string. 见下文中的例子

## 二、拆分与拼接

### 1. str.split方法

#### （a）分割符与str的位置元素选取

In [227]:
s = pd.Series(['a_b_c', 'c_d_e', np.nan, 'f_g_h'], dtype="string")
s

0    a_b_c
1    c_d_e
2     <NA>
3    f_g_h
dtype: string

#### 根据某一个元素分割，默认为空格

In [228]:
s.str.split('_')

0    [a, b, c]
1    [c, d, e]
2         <NA>
3    [f, g, h]
dtype: object

In [230]:
# 由于缺失值的存在, Series对象的 元素级 apply 方法不能使用
#s.apply(lambda x: x.split('_')) 
# AttributeError: 'NAType' object has no attribute 'split'

In [231]:
# 使用apply方法的话,需要分情况--但是 .str 方法为什么能正确执行?
def split_func(x):
    if x is pd.NA:
        y = x
    else :
        y = x.split('_')
    return y
s.apply(lambda x: split_func(x))

0    [a, b, c]
1    [c, d, e]
2         <NA>
3    [f, g, h]
dtype: object

In [233]:
import re
#s.apply(lambda x: re.split(x,'_')) # 使用正则表达式的分割函数同样会因为缺失值导致出问题 TypeError: first argument must be string or compiled pattern

In [248]:
# 使用正则表达式的方法,需重新定义以处理缺失值
def my_re_split(x,rep=''):
    if x is pd.NA:
        y = x
    else :
        y = re.split(rep,x)
    return y
s.apply(lambda x: my_re_split(x,'_'))

0    [a, b, c]
1    [c, d, e]
2         <NA>
3    [f, g, h]
dtype: object

In [244]:
re.split(s[0],'_')

Signature: re.split(pattern, string, maxsplit=0, flags=0)
Source:   
def split(pattern, string, maxsplit=0, flags=0):
    """Split the source string by the occurrences of the pattern,
    returning a list containing the resulting substrings.  If
    capturing parentheses are used in pattern, then the text of all
    groups in the pattern are also returned as part of the resulting
    list.  If maxsplit is nonzero, at most maxsplit splits occur,
    and the remainder of the string is returned as the final element
    of the list."""
    return _compile(pattern, flags).split(string, maxsplit)
File:      c:\programdata\anaconda3\lib\re.py
Type:      function


#### 这里需要注意split后的类型是object，因为现在Series中的元素已经不是string，而包含了list，且string类型只能含有字符串

In [266]:
# 当然,这时候仍然可以用上述的迂回方式将list转为string
s.apply(lambda x: my_re_split(x,'_')).astype('str').astype('string')
# 但实际上很多时候是没必要这么做的,因为我们需要的就是这个list,唯一需要的是要把list分拆成多列

0    ['a', 'b', 'c']
1    ['c', 'd', 'e']
2               <NA>
3    ['f', 'g', 'h']
dtype: string

#### 对于str方法可以进行元素的选择，如果该单元格元素是列表，那么str[i]表示取出第i个元素，如果是单个元素，则先把元素转为列表在取出

In [252]:
s.str.split('_').str[1]

0       b
1       d
2    <NA>
3       g
dtype: object

In [256]:
s.str.split('_')
# 注意分拆后, 非空的单元格里的元素已经是个list了,但str[1]方法能够准确地提取到这个list的相应位置的元素--这就是Series of list对象的向量化切片方法

0    [a, b, c]
1    [c, d, e]
2         <NA>
3    [f, g, h]
dtype: object

In [258]:
# 向量化的切片方式--1
s.str.split('_').str[1:3]

0    [b, c]
1    [d, e]
2      <NA>
3    [g, h]
dtype: object

In [259]:
# 向量化的切片方式--2
# 逆序遍历
s.str.split('_').str[::-1]
# 这么看来, 其他list的各种切片的操作都应该会被这种方法所支持

0    [c, b, a]
1    [e, d, c]
2         <NA>
3    [h, g, f]
dtype: object

In [268]:
# 字符串对象当然也是支持切片的, 但转为字符串以后就不容易取到list的元素了
s.apply(lambda x: my_re_split(x,'_')).astype('str').astype('string').str[:3]
# 一个奇怪的事情发生了--pd.NA对象也被转为 string 了--这不合适...
# .astype('str').astype('string') 会把 pd.NA 也按字面显示转为 string, 这就不合适了吧?

0    ['a
1    ['c
2    <NA
3    ['f
dtype: string

In [273]:
s[2],type(s[2]),

(<NA>, pandas._libs.missing.NAType)

In [276]:
s.apply(lambda x: split_func(x))[2],type(s.apply(lambda x: split_func(x))[2])

(<NA>, pandas._libs.missing.NAType)

In [275]:
s.apply(lambda x: my_re_split(x,'_'))[2],type(s.apply(lambda x: my_re_split(x,'_'))[2]),

(<NA>, pandas._libs.missing.NAType)

In [284]:
s.apply(lambda x: my_re_split(x,'_')).astype('str')[2],type(s.apply(lambda x: my_re_split(x,'_')).astype('str')[2])
# 在这一步就出问题了--astype('str')会按字面显示将objec转为字符串

('<NA>', str)

In [274]:
# pd.NA变成了 string--已经出现了数据处理错误
s.apply(lambda x: my_re_split(x,'_')).astype('str').astype('string')[2],type(s.apply(lambda x: my_re_split(x,'_')).astype('str').astype('string')[2])

('<NA>', str)

In [283]:
pd.Series([1,'1.',pd.NA]).astype('str').astype('string')[2],type(pd.Series([1,'1.',pd.NA]).astype('str').astype('string')[2]) 
pd.Series([1,'1.',np.nan]).astype('str').astype('string')[2],type(pd.Series([1,'1.',np.nan]).astype('str').astype('string')[2]) 
pd.Series([1,'1.',None]).astype('str').astype('string')[2],type(pd.Series([1,'1.',None]).astype('str').astype('string')[2]) 
#pd.Series([1,2,pd.NA]).astype('str').astype('string')[2], type(pd.Series([1,2,pd.NA]).astype('str').astype('string')[2])
#pd.Series([1,2,np.nan]).astype('str').astype('string')[2], type(pd.Series([1,2,np.nan]).astype('str').astype('string')[2])
#pd.Series([True,False]).astype('string')

('None', str)

In [285]:
# astype是个比较霸道的类型转换方式, 缺乏对缺失值的有效操作方式.
s.astype??

Signature: s.astype(dtype, copy: bool = True, errors: str = 'raise') -> ~FrameOrSeries
Source:   
    def astype(
        self: FrameOrSeries, dtype, copy: bool_t = True, errors: str = "raise"
    ) -> FrameOrSeries:
        """
        Cast a pandas object to a specified dtype ``dtype``.

        Parameters
        ----------
        dtype : data type, or dict of column name -> data type
            Use a numpy.dtype or Python type to cast entire pandas object to
            the same type. Alternatively, use {col: dtype, ...}, where col is a
            column label and dtype is a numpy.dtype or Python type to cast one
            or more of the DataFrame's columns to column-specific types.
        copy : bool, default True
            Return a copy when ``copy=True`` (be very careful setting
            ``copy=False`` as changes to values then may propagate to other
            pandas objects).
        errors : {'raise', 'ignore'}, default 'raise'
            Control raising of excep

In [291]:
s.str.split('_')[:][1],  s.str.split('_')[1]# 选取效果和上述是有些区别的

(['c', 'd', 'e'], ['c', 'd', 'e'])

In [294]:
pd.Series(['a_b_c', ['a','b','c']], dtype="object")#.str[1]
#第一个元素先用list('a_b_c')转为['a','_','b','_','c'],然后再切片
#第二个元素是个list, list会正常的切片

0        a_b_c
1    [a, b, c]
dtype: object

In [295]:
pd.Series(['a_b_c', ['a','b','c']], dtype="object").str[1]

0    _
1    b
dtype: object

In [296]:
pd.Series(['a_b_c', ['a','b','c']], dtype="object").str[:2]
# str虽然看起来是转为str再操作, 但实际上, 对于是字符串的单元格,会选择该字符串相应位置的元素,而对于是list的, 则会使用切片方法

0        a_
1    [a, b]
dtype: object

#### （b）其他参数
#### expand参数控制了是否将列拆开，n参数代表最多分割多少次(分割成n+1列)

In [58]:
s.str.split??
#n : int, default -1 (all)
#    Limit number of splits in output.
#    ``None``, 0 and -1 will be interpreted as return all splits.
#expand : bool, default False
#    Expand the splitted strings into separate columns.
#
#    * If ``True``, return DataFrame/MultiIndex expanding dimensionality.
#    * If ``False``, return Series/Index, containing lists of strings.

Signature: s.str.split(pat=None, n=-1, expand=False)
Docstring:
Split strings around given separator/delimiter.

Splits the string in the Series/Index from the beginning,
at the specified delimiter string. Equivalent to :meth:`str.split`.

Parameters
----------
pat : str, optional
    String or regular expression to split on.
    If not specified, split on whitespace.
n : int, default -1 (all)
    Limit number of splits in output.
    ``None``, 0 and -1 will be interpreted as return all splits.
expand : bool, default False
    Expand the splitted strings into separate columns.

    * If ``True``, return DataFrame/MultiIndex expanding dimensionality.
    * If ``False``, return Series/Index, containing lists of strings.

Returns
-------
Series, Index, DataFrame or MultiIndex
    Type matches caller unless ``expand=True`` (see Notes).

See Also
--------
Series.str.split : Split strings around given separator/delimiter.
Series.str.rsplit : Splits string around given separator/delimiter,
  

In [297]:
s.str.split('_',expand=True)

,0,1,2
0,a,b,c
1,c,d,e
2,<NA>,<NA>,<NA>
3,f,g,h


In [301]:
s.str.split('_',n=3) #expand 默认是 False, 因此单独指定 n=3 是没有效果的

0    [a, b, c]
1    [c, d, e]
2         <NA>
3    [f, g, h]
dtype: object

In [302]:
s.str.split('_',expand=True,n=1) # n=1 则拆分一次, expand=True 则会拆分成多列--由于只进行了一次拆分,于是被拆分成了1+1=2列

,0,1
0,a,b_c
1,c,d_e
2,<NA>,<NA>
3,f,g_h


In [303]:
s.str.split('_',expand=True,n=1)

,0,1
0,a,b_c
1,c,d_e
2,<NA>,<NA>
3,f,g_h


### 2. str.cat方法

#### （a）不同对象的拼接模式
#### cat方法对于不同对象的作用结果并不相同，其中的对象包括：单列、双列、多列

#### ① 对于单个Series而言，就是指所有的元素进行字符合并为一个字符串

In [304]:
s = pd.Series(['ab',None,'d'],dtype='string')
s

0      ab
1    <NA>
2       d
dtype: string

In [305]:
s.str.cat()

'abd'

#### 其中可选sep分隔符参数，和缺失值替代字符na_rep参数

In [306]:
s.str.cat(sep=',')
# oracle 里有 listagg(11.2及以上)和 wm_concat 两个函数实现相同功能

'ab,d'

In [307]:
s.str.cat(sep=',',na_rep='*')

'ab,*,d'

#### ② 对于两个Series合并而言，是对应索引的元素进行合并

In [308]:
s2 = pd.Series(['24','cc',None],dtype='string')
s2

0      24
1      cc
2    <NA>
dtype: string

In [309]:
s.str.cat(s2)
# 第二和第三个元素都成了pd.NA了--默认情况下,缺失值会传播

0    ab24
1    <NA>
2    <NA>
dtype: string

#### 同样也有相应参数，需要注意的是两个缺失值会被同时替换

In [310]:
s.str.cat(s2,sep=',',na_rep='*')

0    ab,24
1     *,cc
2      d,*
dtype: string

#### ③ 多列拼接可以分为表的拼接和多Series拼接

#### 表的拼接

In [84]:
display(s)
d_ = pd.DataFrame({0:['1','3','5'],1:['5','b',None]},dtype='string')
display(d_)
s.str.cat(d_,na_rep='*')
# 将一个 Series 对象和一个等长的 DF 对象按索引进行行的拼接.

0      ab
1    <NA>
2       d
dtype: string

,0,1
0,1,5
1,3,b
2,5,<NA>


0    ab15
1     *3b
2     d5*
dtype: string

In [311]:
s.to_frame().join(d_,lsuffix='_l')

,0_l,0,1
0,ab,1,5
1,<NA>,3,b
2,d,5,<NA>


In [313]:
# s.to_frame().join(d_,lsuffix='_l').str.cat(na_rep='*') 
#AttributeError: 'DataFrame' object has no attribute 'str'
# 想把 DF 的行拼接到一起还不是那么直接

In [316]:
# 一个将df按行拼接起来的思路
# 先构造索引一致且值全为空字符串的Series,然后用Series拼接df的方式, 用这个Series去拼接表
df=pd.DataFrame(np.random.randn(21).reshape(3,7),index=list('abc'),columns=['C_1','C_2','C_3','C_4','C_5','C_6','C_7']).astype('str')
display(df)
s1=pd.Series(''*3,index=df.index)
display(s1)
s1.str.cat(df,na_rep='*')

,C_1,C_2,C_3,C_4,C_5,C_6,C_7
a,-0.48579020425720065,-0.35535860921712165,1.4294993315771587,-0.30434309277741795,0.6839930283234016,0.842549343534401,0.9458261107264027
b,-0.812637155402051,1.3044365527815767,1.7906639459751705,0.5540285184142737,-0.7905375926250731,1.3407465660541444,-0.12854770358914622
c,-0.5048545900626982,-0.10944741852284397,-0.2252323595600378,0.773199652481586,-0.1830636381161688,0.7660927164744896,-1.1427259867401522


a    
b    
c    
dtype: object

a    -0.48579020425720065-0.355358609217121651.4294...
b    -0.8126371554020511.30443655278157671.79066394...
c    -0.5048545900626982-0.10944741852284397-0.2252...
dtype: object

#### 多个Series按索引拼接

In [319]:
s.str.cat([s+'0',s*2])

0    abab0abab
1         <NA>
2        dd0dd
dtype: string

In [320]:
#有没有类似的顶级方法?

#### （b）cat中的索引对齐
#### 当前版本中，如果两边合并的索引不相同且未指定join参数，默认为左连接，设置join='left'

In [321]:
s2 = pd.Series(list('abc'),index=[1,2,3],dtype='string')
s2

1    a
2    b
3    c
dtype: string

In [322]:
s.str.cat(s2,na_rep='*')

0    ab*
1     *a
2     db
dtype: string

In [323]:
# 如果二者索引没有对齐,相当于左连接,然后再拼接--由于左连接可能会引入缺失值,如果索引完全不对齐则会把s的每个元素与一个na_rep参数指定的字符串拼接起来.
s.str.cat(s1,na_rep='*')

0    ab*
1     **
2     d*
dtype: string

In [324]:
# 不指定na_rep参数,则会导致缺失值的扩散
s.str.cat(s1)

0    <NA>
1    <NA>
2    <NA>
dtype: string

In [325]:
s1,s

(a    
 b    
 c    
 dtype: object,
 0      ab
 1    <NA>
 2       d
 dtype: string)

In [326]:
# 更换s1和s2的位置, 发现确实是左连接
s1.str.cat(s,na_rep='*')

a    *
b    *
c    *
dtype: object

## 三、替换
#### 广义上的替换，就是指str.replace函数的应用，fillna是针对缺失值的替换，上一章已经提及
#### 提到替换，就不可避免地接触到正则表达式，这里默认读者已掌握常见正则表达式知识点，若对其还不了解的，可以通过[这份资料](https://regexone.com/)来熟悉

### 1. str.replace的常见用法

In [328]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca','', np.nan, 'CABA', 'dog', 'cat'],dtype="string")
s

0       A
1       B
2       C
3    Aaba
4    Baca
5        
6    <NA>
7    CABA
8     dog
9     cat
dtype: string

#### 第一个值写r开头的正则表达式，后一个写替换的字符串

In [329]:
s.str.replace(r'^[AB]','***') #把字符串开头的A或B(大写)替换为"***"

0       ***
1       ***
2         C
3    ***aba
4    ***aca
5          
6      <NA>
7      CABA
8       dog
9       cat
dtype: string

### 2. 子组与函数替换

#### 通过正整数调用子组（0返回字符本身，从1开始才是子组）

In [106]:
s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2)[1:]+'*')
#repl : str or callable
#    Replacement string or a callable. The callable is passed the regex
#    match object and must return a replacement string to be used.
#    See :func:`re.sub`.

0       A
1       B
2       C
3     ba*
4     ca*
5        
6    <NA>
7     BA*
8     dog
9     cat
dtype: string

In [330]:
s

0       A
1       B
2       C
3    Aaba
4    Baca
5        
6    <NA>
7    CABA
8     dog
9     cat
dtype: string

In [332]:
s.str.replace(r'([ABC])(\w+)',lambda x:x.group(0))#[0:]+'*')
# group(0) 返回字符串本身--没有匹配到的也是返回整个字符串, 如果是缺失值则还是返回缺失值

0       A
1       B
2       C
3    Aaba
4    Baca
5        
6    <NA>
7    CABA
8     dog
9     cat
dtype: string

In [333]:
s.str.replace(r'([ABC])(\w+)',lambda x:x.group(1))#[0:]+'*')
# group(1) 返回匹配到的第一个子组--没有匹配的则继续返回整个字符串

0       A
1       B
2       C
3       A
4       B
5        
6    <NA>
7       C
8     dog
9     cat
dtype: string

In [334]:
s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2))#[0:]+'*')
# group(2) 返回匹配到的第二个子组--没有匹配的则继续返回整个字符串
# 特别地,只匹配到一个子组的, 返回的是第一个子组呢,还是整个字符串呢?

0       A
1       B
2       C
3     aba
4     aca
5        
6    <NA>
7     ABA
8     dog
9     cat
dtype: string

In [376]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca','', np.nan, 'CABA', 'dog', 'cat'],dtype="string")
dfs=s.to_frame()
dfs["s.str.replace(r'([ABC])(\w+)',lambda x:x.group(0))"]=s.str.replace(r'([ABC])(\w+)',lambda x:x.group(0))
dfs["s.str.replace(r'([ABC])(\w+)',lambda x:x.group(1))"]=s.str.replace(r'([ABC])(\w+)',lambda x:x.group(1))
dfs["s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2))"]=s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2))
#dfs["s.str.replace(r'([ABC])(\w+)',lambda x:x.group(3))"]=s.str.replace(r'([ABC])(\w+)',lambda x:x.group(3)) # IndexError: no such group
#dfs["s.str.replace(r'([ABC])(\w+)',lambda x:x.group(4))"]=s.str.replace(r'([ABC])(\w+)',lambda x:x.group(4))
dfs["s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2)[1:])"]=s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2)[1:])
dfs["s.str.replace(r'([ABC])(\w+)',lambda x:x.group(0)[1:]+'****')"]=s.str.replace(r'([ABC])(\w+)',lambda x:x.group(0)[1:]+'****')#012689列都没有拼接****,但是有返回值
dfs["s.str.replace(r'([ABC])(\w+)',lambda x:x.group(1)[1:]+'****')"]=s.str.replace(r'([ABC])(\w+)',lambda x:x.group(1)[1:]+'****')
dfs["s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2)[1:]+'****')"]=s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2)[1:]+'****')
dfs.T

,0,1,2,3,4,5,6,7,8,9
0,A,B,C,Aaba,Baca,,<NA>,CABA,dog,cat
"s.str.replace(r'([ABC])(\w+)',lambda x:x.group(0))",A,B,C,Aaba,Baca,,<NA>,CABA,dog,cat
"s.str.replace(r'([ABC])(\w+)',lambda x:x.group(1))",A,B,C,A,B,,<NA>,C,dog,cat
"s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2))",A,B,C,aba,aca,,<NA>,ABA,dog,cat
"s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2)[1:])",A,B,C,ba,ca,,<NA>,BA,dog,cat
"s.str.replace(r'([ABC])(\w+)',lambda x:x.group(0)[1:]+'****')",A,B,C,aba****,aca****,,<NA>,ABA****,dog,cat
"s.str.replace(r'([ABC])(\w+)',lambda x:x.group(1)[1:]+'****')",A,B,C,****,****,,<NA>,****,dog,cat
"s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2)[1:]+'****')",A,B,C,ba****,ca****,,<NA>,BA****,dog,cat


In [374]:
# 还需进一步研究...............
# 把ss的第一个元素设置为
ss = pd.Series(['Aa', 'B', 'C', 'Aaba', 'Baca','', np.nan, 'CABA', 'dog', 'cat'],dtype="string")
ss.str.replace('([ABC])([abc])',lambda x:x.group(0))#[0:]+'*')


0      Aa
1       B
2       C
3    Aaba
4    Baca
5        
6    <NA>
7    CABA
8     dog
9     cat
dtype: string

In [107]:
s.str.replace??

Signature: s.str.replace(pat, repl, n=-1, case=None, flags=0, regex=True)
Docstring:
Replace occurrences of pattern/regex in the Series/Index with
some other string. Equivalent to :meth:`str.replace` or
:func:`re.sub`.

Parameters
----------
pat : str or compiled regex
    String can be a character sequence or regular expression.
repl : str or callable
    Replacement string or a callable. The callable is passed the regex
    match object and must return a replacement string to be used.
    See :func:`re.sub`.
n : int, default -1 (all)
    Number of replacements to make from start.
case : bool, default None
    Determines if replace is case sensitive:

    - If True, case sensitive (the default if `pat` is a string)
    - Set to False for case insensitive
    - Cannot be set if `pat` is a compiled regex.

flags : int, default 0 (no flags)
    Regex module flags, e.g. re.IGNORECASE. Cannot be set if `pat` is a compiled
    regex.
regex : bool, default True
    Determines if assumes the 

In [105]:
re.sub??

Signature: re.sub(pattern, repl, string, count=0, flags=0)
Source:   
def sub(pattern, repl, string, count=0, flags=0):
    """Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the Match object and must return
    a replacement string to be used."""
    return _compile(pattern, flags).sub(repl, string, count)
File:      c:\programdata\anaconda3\lib\re.py
Type:      function


#### 利用?P<....>表达式可以对子组命名调用

In [377]:
s.str.replace(r'(?P<one>[ABC])(?P<two>\w+)',lambda x:x.group('two')[1:]+'*')

0       A
1       B
2       C
3     ba*
4     ca*
5        
6    <NA>
7     BA*
8     dog
9     cat
dtype: string

In [378]:
s.str.replace(r'(?P<one>[ABC])(?P<two>\w+)',lambda x:print(x)) # lambda 中的 x 表示前边的正则表达式匹配到的子串

<re.Match object; span=(0, 4), match='Aaba'>
<re.Match object; span=(0, 4), match='Baca'>
<re.Match object; span=(0, 4), match='CABA'>


0       A
1       B
2       C
3        
4        
5        
6    <NA>
7        
8     dog
9     cat
dtype: string

In [379]:
s

0       A
1       B
2       C
3    Aaba
4    Baca
5        
6    <NA>
7    CABA
8     dog
9     cat
dtype: string

In [380]:
ss=s+'FF123'
ss

0       AFF123
1       BFF123
2       CFF123
3    AabaFF123
4    BacaFF123
5        FF123
6         <NA>
7    CABAFF123
8     dogFF123
9     catFF123
dtype: string

In [381]:
ss.str.replace(r'(?P<one>[ABC])(?P<two>\w+)',lambda x:print(x.group(0),x.group(1),x.group(2))) # lambda 中的 x 表示前边的正则表达式匹配到的子串

AFF123 A FF123
BFF123 B FF123
CFF123 C FF123
AabaFF123 A abaFF123
BacaFF123 B acaFF123
CABAFF123 C ABAFF123


0            
1            
2            
3            
4            
5       FF123
6        <NA>
7            
8    dogFF123
9    catFF123
dtype: string

### 3. 关于str.replace的注意事项
#### 首先，要明确str.replace和replace并不是一个东西：
#### str.replace针对的是object类型或string类型，默认是以正则表达式为操作，目前暂时不支持DataFrame上使用
#### replace针对的是任意类型的序列或数据框，如果要以正则表达式替换，需要设置regex=True，该方法通过字典可支持多列替换
#### 但现在由于string类型的初步引入，用法上出现了一些问题，这些issue有望在以后的版本中修复


#### （a）str.replace赋值参数不得为pd.NA
#### 这听上去非常不合理，例如对满足某些正则条件的字符串替换为缺失值，直接更改为缺失值在当下版本就会报错

In [164]:
#pd.Series(['A','B'],dtype='string').str.replace(r'[A]',pd.NA) #报错 TypeError: repl must be a string or callable
#pd.Series(['A','B'],dtype='O').str.replace(r'[A]',pd.NA) #报错 TypeError: repl must be a string or callable

#### 此时，可以先转为object类型再转换回来，曲线救国：

In [382]:
pd.Series(['A','B'],dtype='string').astype('O').replace(r'[A]',pd.NA,regex=True).astype('string')

0    <NA>
1       B
dtype: string

#### 至于为什么不用replace函数的regex替换（但string类型replace的非正则替换是可以的），原因在下面一条

#### （b）对于string类型Series，在使用replace函数时不能使用正则表达式替换
#### 该bug现在还未修复

In [383]:
pd.Series(['A','B'],dtype='string').replace(r'[A]','C',regex=True)

0    A
1    B
dtype: string

In [384]:
pd.Series(['A','B'],dtype='O').replace(r'[A]','C',regex=True)

0    C
1    B
dtype: object

#### （c）string类型序列如果存在缺失值，不能使用replace替换

In [385]:
pd.Series(['A',np.nan],dtype='string').replace('A','B') #报错--没有报错啊

0       B
1    <NA>
dtype: string

In [389]:
pd.Series(['A',np.nan],dtype='str').replace('A','B') # str 类型也没有报错
pd.Series(['A',np.nan],dtype='O').replace('A','B') # O 类型也没有报错

0      B
1    NaN
dtype: object

In [386]:
pd.Series(['A',np.nan],dtype='string').str.replace('A','B')

0       B
1    <NA>
dtype: string

#### 综上，概况的说，除非需要赋值元素为缺失值（转为object再转回来），否则请使用str.replace方法

## 四、子串匹配与提取

### 1. str.extract方法
从单元格里提取符合传入的正则表达式的子串, 也可以用来将一列扩张为多列.

#### （a）常见用法

In [390]:
display(pd.Series(['10-87', '10-88', '10-89'],dtype="string"))
pd.Series(['10-87', '10-88', '10-89'],dtype="string").str.extract(r'([\d]{2})-([\d]{2})')

0    10-87
1    10-88
2    10-89
dtype: string

,0,1
0,10,87
1,10,88
2,10,89


In [176]:
s.str.extract??

Signature: s.str.extract(pat, flags=0, expand=True)
Docstring:
Extract capture groups in the regex `pat` as columns in a DataFrame.

For each subject string in the Series, extract groups from the
first match of regular expression `pat`.

Parameters
----------
pat : str
    Regular expression pattern with capturing groups.
flags : int, default 0 (no flags)
    Flags from the ``re`` module, e.g. ``re.IGNORECASE``, that
    modify regular expression matching for things like case,
    spaces, etc. For more details, see :mod:`re`.
expand : bool, default True
    If True, return DataFrame with one column per capture group.
    If False, return a Series/Index if there is one capture group
    or DataFrame if there are multiple capture groups.

Returns
-------
DataFrame or Series or Index
    A DataFrame with one row for each subject string, and one
    column for each group. Any capture group names in regular
    expression pat will be used for column names; otherwise
    capture group number

#### 通过在正则表达式中使用子组名来为拆分后的列增加列名

In [391]:
pd.Series(['10-87', '10-88', '-89'],dtype="string").str.extract(r'(?P<start>[\d]{2})-(?P<end>[\d]{2})')

,start,end
0,10,87
1,10,88
2,<NA>,<NA>


#### 利用?正则标记选择部分提取

In [392]:
pd.Series(['10-87', '10-88', '10-',  '-89'],dtype="string").str.extract(r'(?P<name_1>[\d]{2})?-(?P<name_2>[\d]{2})?')
# 第一组后的?表示该组是可能缺失的,第一组没匹配到的也能够正确提取第二组.
# 在第二组后加上?则第二组匹配不到的也会正确提取第一组.

,name_1,name_2
0,10,87
1,10,88
2,10,<NA>
3,<NA>,89


In [393]:
pd.Series(['10-87', '10-88', '10-'],dtype="string").str.extract(r'(?P<name_1>[\d]{2})-(?P<name_2>[\d]{2})?')

,name_1,name_2
0,10,87
1,10,88
2,10,<NA>


#### （b）expand参数（默认为True）

#### 对于一个子组的Series，如果expand设置为False，则返回Series，若大于一个子组，则expand参数无效，全部返回DataFrame
#### 对于一个子组的Index，如果expand设置为False，则返回提取后的Index，若大于一个子组且expand为False，报错

In [394]:
s = pd.Series(["a1", "b2", "c3"], ["A11", "B22", "C33"], dtype="string")
s#.index

A11    a1
B22    b2
C33    c3
dtype: string

In [395]:
s.str.extract(r'([\w])')

,0
A11,a
B22,b
C33,c


In [396]:
s.str.extract(r'([\w])',expand=False)

A11    a
B22    b
C33    c
dtype: string

In [397]:
s.str.extract(r'([\w])',expand=True)

,0
A11,a
B22,b
C33,c


In [398]:
('m'+s).str.extract(r'([\w])([\w])([\w])',expand=False) # 大于1个组, 因此 expand=False 参数就没法起作用了--这和split方法中需要先指定expand, n才能起作用是不一致的.

,0,1,2
A11,m,a,1
B22,m,b,2
C33,m,c,3


In [399]:
s.index.str.extract(r'([\w])')

,0
0,A
1,B
2,C


In [400]:
s.index.str.extract(r'([\w])',expand=False)

Index(['A', 'B', 'C'], dtype='object')

In [401]:
# 提取两个子组类型,都提取导了,因此自然会分成两列
s.index.str.extract(r'([\w])([\d])')

,0,1
0,A,1
1,B,2
2,C,3


In [409]:
# s.index.str.extract(r'([\w])([\d])([\w])') # 第三个子组会继续沿用第二个
# s.index.str.extract(r'([\w])([\d])([\d])') # 同上
s.index.str.extract(r'([\w])([\d])([\w])([\w])') #再增加一个就都变成 NaN 了

,0,1,2,3
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN


In [413]:
s.index.str.extract(r'([\w])([\d])([\d])')#([\d])') #三个没问题, 再增加一个就都变成 NaN 了

,0,1,2
0,A,1,1
1,B,2,2
2,C,3,3


In [415]:
# 索引也支持提取
#s.index.str.extract(r'([\w])([\d])',expand=False) #报错 ValueError: only one regex group is supported with Index
s.index.str.extract(r'([\w])([\d])',expand=True)

,0,1
0,A,1
1,B,2
2,C,3


In [416]:
# 对于索引, 也可以先使用 to_series 函数(注意s小写)转为Series再提取--这在从复合型单层索引构造层次索引时会用到.
s.index.to_series().str.extract(r'([\w])([\d])',expand=False)

,0,1
A11,A,1
B22,B,2
C33,C,3


### 2. str.extractall方法

#### 与extract只匹配第一个符合条件的表达式不同，extractall会找出所有符合条件的字符串，并建立多级索引（即使只找到一个）

In [417]:
s = pd.Series(["a1a2", "b1", "c1"], index=["A", "B", "C"],dtype="string")
display(s)
two_groups = '(?P<letter>[a-z])(?P<digit>[0-9])'
s.str.extract(two_groups, expand=True)

A    a1a2
B      b1
C      c1
dtype: string

,letter,digit
A,a,1
B,b,1
C,c,1


In [418]:
s.str.extractall(two_groups)
# 将匹配到的多个符合条件的子串分为了多行--分为列的话会导致大量的单元格是缺失值

letter digit
  match             
A 0          a     1
  1          a     2
B 0          b     1
C 0          c     1

In [47]:
s['A']='a1'
s.str.extractall(two_groups)

,,letter,digit
,match,,
A,0,a,1
B,0,b,1
C,0,c,1


#### 如果想查看第i层匹配，可使用xs方法

In [48]:
s = pd.Series(["a1a2", "b1b2", "c1c2"], index=["A", "B", "C"],dtype="string")
s.str.extractall(two_groups).xs(1,level='match')

,letter,digit
A,a,2
B,b,2
C,c,2


### 3. str.contains和str.match
#### 前者的作用为检测是否包含某种正则模式

In [49]:
pd.Series(['1', None, '3a', '3b', '03c'], dtype="string").str.contains(r'[0-9][a-z]')

0    False
1     <NA>
2     True
3     True
4     True
dtype: boolean

#### 可选参数为na

In [50]:
pd.Series(['1', None, '3a', '3b', '03c'], dtype="string").str.contains('a', na=False)

0    False
1    False
2     True
3    False
4    False
dtype: boolean

#### str.match与其区别在于，match依赖于python的re.match，检测内容为是否从头开始包含该正则模式

In [51]:
pd.Series(['1', None, '3a_', '3b', '03c'], dtype="string").str.match(r'[0-9][a-z]',na=False)

0    False
1    False
2     True
3     True
4    False
dtype: boolean

In [52]:
pd.Series(['1', None, '_3a', '3b', '03c'], dtype="string").str.match(r'[0-9][a-z]',na=False)

0    False
1    False
2    False
3     True
4    False
dtype: boolean

## 五、常用字符串方法

### 1. 过滤型方法
#### （a）str.strip
#### 常用于过滤空格

In [53]:
pd.Series(list('abc'),index=[' space1  ','space2  ','  space3'],dtype="string").index.str.strip()

Index(['space1', 'space2', 'space3'], dtype='object')

#### （b）str.lower和str.upper

In [54]:
pd.Series('A',dtype="string").str.lower()

0    a
dtype: string

In [55]:
pd.Series('a',dtype="string").str.upper()

0    A
dtype: string

#### （c）str.swapcase和str.capitalize
#### 分别表示交换字母大小写和大写首字母

In [56]:
pd.Series('abCD',dtype="string").str.swapcase()

0    ABcd
dtype: string

In [57]:
pd.Series('abCD',dtype="string").str.capitalize()

0    Abcd
dtype: string

### 2. isnumeric方法
#### 检查每一位是否都是数字，请问如何判断是否是数值？（问题二）

In [422]:
pd.Series(['1.2','1','-0.3','a',np.nan],dtype="string").str.isnumeric()

0    False
1     True
2    False
3    False
4     <NA>
dtype: boolean

## 六、问题与练习
### 1. 问题

#### 【问题一】 str对象方法和df/Series对象方法有什么区别？

#### 【问题二】 给出一列string类型，如何判断单元格是否是数值型数据？

In [426]:
s_string = pd.Series(['1.2','1','-0.3','a',np.nan],dtype="string")
dsiplay(s_string)# .str.extract(r'\d')  #ValueError: pattern contains no capture groups

0     1.2
1       1
2    -0.3
3       a
4    <NA>
dtype: string

In [438]:
s_string.str.isdecimal()

0    False
1     True
2    False
3    False
4     <NA>
dtype: boolean

In [441]:
s_string.apply(lambda x:print(x,type(x)))

1.2 <class 'str'>
1 <class 'str'>
-0.3 <class 'str'>
a <class 'str'>
<NA> <class 'pandas._libs.missing.NAType'>


0    None
1    None
2    None
3    None
4    None
dtype: object

#### 【问题三】 rsplit方法的作用是什么？它在什么场合下适用？

In [443]:
s.str.rsplit??

Signature: s.str.rsplit(pat=None, n=-1, expand=False)
Docstring:
Split strings around given separator/delimiter.

Splits the string in the Series/Index from the end,
at the specified delimiter string. Equivalent to :meth:`str.rsplit`.

Parameters
----------
pat : str, optional
    String or regular expression to split on.
    If not specified, split on whitespace.
n : int, default -1 (all)
    Limit number of splits in output.
    ``None``, 0 and -1 will be interpreted as return all splits.
expand : bool, default False
    Expand the splitted strings into separate columns.

    * If ``True``, return DataFrame/MultiIndex expanding dimensionality.
    * If ``False``, return Series/Index, containing lists of strings.

Returns
-------
Series, Index, DataFrame or MultiIndex
    Type matches caller unless ``expand=True`` (see Notes).

See Also
--------
Series.str.split : Split strings around given separator/delimiter.
Series.str.rsplit : Splits string around given separator/delimiter,
    st

#### 【问题四】 在本章的第二到第四节分别介绍了字符串类型的5类操作，请思考它们各自应用于什么场景？

* 拆分与拼接
    * str.split--拆分
    * str.cat--拼接
* 替换
* 子串匹配与提取
    * extract--从pandas对象的一列中提取信息成为新的列,例如将手机号码,邮件从文本中提取出来.
    * exractall--提取所有能够匹配到的子串,并建立多级索引
    * contain--判断包含某个特定格式的子串
    * match--字符串以某个特定子串开头
* 过滤与判断

In [473]:
# 将文末参考文献按字段整理成表格形式
!type data\导出题录.txt
# 中文字符, 需要指定encoding

[1]鑲栭箯,褰�鍡ｇ��,鐜嬭暰. 鍩烘湰鍘熷垯涓庡叧閿�闂�棰樷�斺�斿�︽湳鍨嬪浘涔﹂�嗛�嗗憳濡備綍鍚�鍔ㄦ暟瀛椾汉鏂囬」鐩甗J]. 鍥句功棣嗚�哄潧, 2017, 鍗风己澶�(鏈熺己澶�): 20-25.
[2]寮犺埖,鍚磋穬浼�. 鍥藉�栧浘涔﹂�嗘敮鎸佹暟瀛椾汉鏂囩殑瀹炶返鍙婂惎绀篬J]. 鍥句功棣嗘潅蹇�, 2014, 鍗风己澶�(鏈熺己澶�): 47-52.
[3]鍒樼倻,璋㈣搲,寮犵��,绛�. 闈㈠悜浜烘枃鐮旂┒鐨勫浗瀹舵暟鎹�鍩虹��璁炬柦寤鸿�綶J]. 涓�鍥藉浘涔﹂�嗗�︽姤, 2016, 鍗风己澶�(鏈熺己澶�): 29-39.
[4]鍐�鏅�,闄堟儬鍏�. 鍥藉�栧浘涔﹂�嗗弬涓庢暟瀛椾汉鏂囩爺绌惰堪璇刐J]. 鍥句功棣嗘潅蹇�, 2016, 鍗风己澶�(鏈熺己澶�): 14-19.
[5]楂樿儨瀵�,璧靛畤缈�,鏈卞簡鍗�. 鍥藉唴澶栨暟瀛椾汉鏂囬�嗗煙鐮旂┒杩涘睍鍒嗘瀽[J]. 鍥句功棣嗘潅蹇�, 2016, 鍗风己澶�(鏈熺己澶�): 9-18.
[6]鏇捐暰,鐜嬫檽鍏�,鑼冪倻. 鍥炬。鍗氶�嗗煙鐨勬櫤鎱ф暟鎹�鍙婂叾鍦ㄦ暟瀛椾汉鏂囩爺绌朵腑鐨勮�掕壊[J]. 涓�鍥藉浘涔﹂�嗗�︽姤, 2018, 鍗风己澶�(鏈熺己澶�): 17-34.
[7]璧电敓杈�,鏈卞�﹁姵. 鎴戝浗楂樻牎鏁板瓧浜烘枃涓�蹇冨缓璁惧垵鎺�[J]. 鍥句功鎯呮姤宸ヤ綔, 2014, 鍗风己澶�(鏈熺己澶�): 64-69, 100.
[8]鐔婅帀鍚�,寮犵�忛槼,寮犵伩. 鍥句功棣嗗湪鏁板瓧浜烘枃棰嗗煙鐨勪紶鎾�鍔熻兘涓庢湇鍔＄爺绌禰J]. 鍥句功棣�, 2016, 鍗风己澶�(鏈熺己澶�): 88-93, 99.
[9]閭撹�佺劧,鏉庡皯璐�. 缇庡浗楂樻牎鏁板瓧浜烘枃涓�蹇冭皟鏌�[J]. 鍥句功棣嗚�哄潧, 2016, 鍗风己澶�(鏈熺己澶�): 26-34.
[10]鏈辨湰鍐�,鑱傚崕. 璺ㄧ晫涓庤瀺鍚�:鍏ㄧ悆瑙嗛噹涓嬬殑鏁板瓧浜烘枃鈥斺�旈�栧眾鍖椾含澶у�︹�滄暟瀛椾汉鏂囪�哄潧鈥濅細璁�缁艰堪[J]. 澶у�﹀浘涔﹂�嗗�︽姤, 2016, 鍗风己澶�(鏈熺己澶�): 16-21.
[11]宸﹀��,寮犲崼涓�. 鏁板瓧浜烘枃瑙嗚�掍笅鐨勬。妗堝�︾爺绌禰J]. 鍥句功涓庢儏鎶�, 2019, 鍗风己澶�(鏈熺己澶�): 94-100

In [541]:
# 第一步, 指定编码,读取文本
ref=open('data\导出题录.txt',encoding='utf8').read()
print(ref)
# 第二步,按换行符分割后转为Series对象,以便于使用Series的 .str 类方法
refs=pd.Series(ref.split('\n'))
# 第三步,拆分
##第一次拆分,使用 .
refs=refs.str.split(r'.',expand=True)
##第二次拆分,使用 , 将未分拆的列进一步拆分,并与其他列join
refs=refs[[0,1]].join(refs[2].str.split(',',expand=True),rsuffix='_')
##第三次拆分,选取有用的非空列,将未拆分的列进一步拆分,并于其他有用的列join
refs=refs[['0','1','0_','1_']].join(refs[2].str.split(':',expand=True),rsuffix='_')
# 第四步,丢掉空行
refs=refs[['0','1','0_','1_',1]].dropna()
# 第五步,为拆分后的列添加列名
refs.columns=['作者','标题','期刊','年','页码']
refs

[1]肖鹏,彭嗣禹,王蕾. 基本原则与关键问题——学术型图书馆馆员如何启动数字人文项目[J]. 图书馆论坛, 2017, 卷缺失(期缺失): 20-25.
[2]张舵,吴跃伟. 国外图书馆支持数字人文的实践及启示[J]. 图书馆杂志, 2014, 卷缺失(期缺失): 47-52.
[3]刘炜,谢蓉,张磊,等. 面向人文研究的国家数据基础设施建设[J]. 中国图书馆学报, 2016, 卷缺失(期缺失): 29-39.
[4]冯晴,陈惠兰. 国外图书馆参与数字人文研究述评[J]. 图书馆杂志, 2016, 卷缺失(期缺失): 14-19.
[5]高胜寒,赵宇翔,朱庆华. 国内外数字人文领域研究进展分析[J]. 图书馆杂志, 2016, 卷缺失(期缺失): 9-18.
[6]曾蕾,王晓光,范炜. 图档博领域的智慧数据及其在数字人文研究中的角色[J]. 中国图书馆学报, 2018, 卷缺失(期缺失): 17-34.
[7]赵生辉,朱学芳. 我国高校数字人文中心建设初探[J]. 图书情报工作, 2014, 卷缺失(期缺失): 64-69, 100.
[8]熊莉君,张福阳,张灿. 图书馆在数字人文领域的传播功能与服务研究[J]. 图书馆, 2016, 卷缺失(期缺失): 88-93, 99.
[9]邓要然,李少贞. 美国高校数字人文中心调查[J]. 图书馆论坛, 2016, 卷缺失(期缺失): 26-34.
[10]朱本军,聂华. 跨界与融合:全球视野下的数字人文——首届北京大学“数字人文论坛”会议综述[J]. 大学图书馆学报, 2016, 卷缺失(期缺失): 16-21.
[11]左娜,张卫东. 数字人文视角下的档案学研究[J]. 图书与情报, 2019, 卷缺失(期缺失): 94-100.
[12]朱娜. 数字人文的兴起及图书馆的角色[J]. 图书馆, 2016, 卷缺失(期缺失): 17-22, 48.
[13]唐乐. 耶鲁大学图书馆的数字人文服务实践[J]. 图书馆论坛, 2018, 卷缺失(期缺失): 10-18.
[14]朱本军,聂华. 数字人文:图书馆实践的新方向[J]. 大学图书馆学报, 2017, 卷缺失(期缺失): 23-29.
[15]曾小莹. 数字人文背景下的图书馆:作用与服务[J]. 图书与情报, 2014, 卷缺失(期缺失): 111-113.
[1

,作者,标题,期刊,年,页码
0,"[1]肖鹏,彭嗣禹,王蕾",基本原则与关键问题——学术型图书馆馆员如何启动数字人文项目[J],图书馆论坛,2017,20-25
1,"[2]张舵,吴跃伟",国外图书馆支持数字人文的实践及启示[J],图书馆杂志,2014,47-52
2,"[3]刘炜,谢蓉,张磊,等",面向人文研究的国家数据基础设施建设[J],中国图书馆学报,2016,29-39
3,"[4]冯晴,陈惠兰",国外图书馆参与数字人文研究述评[J],图书馆杂志,2016,14-19
4,"[5]高胜寒,赵宇翔,朱庆华",国内外数字人文领域研究进展分析[J],图书馆杂志,2016,9-18
5,"[6]曾蕾,王晓光,范炜",图档博领域的智慧数据及其在数字人文研究中的角色[J],中国图书馆学报,2018,17-34
6,"[7]赵生辉,朱学芳",我国高校数字人文中心建设初探[J],图书情报工作,2014,64-69
7,"[8]熊莉君,张福阳,张灿",图书馆在数字人文领域的传播功能与服务研究[J],图书馆,2016,88-93
8,"[9]邓要然,李少贞",美国高校数字人文中心调查[J],图书馆论坛,2016,26-34
9,"[10]朱本军,聂华",跨界与融合:全球视野下的数字人文——首届北京大学“数字人文论坛”会议综述[J],大学图书馆学报,2016,16-21


In [542]:
# 再使用 srt.replace 方法将各列里无关的信息用正则表达式替换掉--作者
refs.作者=refs.作者.str.replace(r'\[\d+\]','')
refs
refs.标题=refs.标题.str.replace(r'\[J\]','')
refs

,作者,标题,期刊,年,页码
0,"肖鹏,彭嗣禹,王蕾",基本原则与关键问题——学术型图书馆馆员如何启动数字人文项目,图书馆论坛,2017,20-25
1,"张舵,吴跃伟",国外图书馆支持数字人文的实践及启示,图书馆杂志,2014,47-52
2,"刘炜,谢蓉,张磊,等",面向人文研究的国家数据基础设施建设,中国图书馆学报,2016,29-39
3,"冯晴,陈惠兰",国外图书馆参与数字人文研究述评,图书馆杂志,2016,14-19
4,"高胜寒,赵宇翔,朱庆华",国内外数字人文领域研究进展分析,图书馆杂志,2016,9-18
5,"曾蕾,王晓光,范炜",图档博领域的智慧数据及其在数字人文研究中的角色,中国图书馆学报,2018,17-34
6,"赵生辉,朱学芳",我国高校数字人文中心建设初探,图书情报工作,2014,64-69
7,"熊莉君,张福阳,张灿",图书馆在数字人文领域的传播功能与服务研究,图书馆,2016,88-93
8,"邓要然,李少贞",美国高校数字人文中心调查,图书馆论坛,2016,26-34
9,"朱本军,聂华",跨界与融合:全球视野下的数字人文——首届北京大学“数字人文论坛”会议综述,大学图书馆学报,2016,16-21


In [545]:
# 再拼接成参考文献样式
refs=refs.reset_index()
refs.head()

,index,作者,标题,期刊,年,页码
0,0,"肖鹏,彭嗣禹,王蕾",基本原则与关键问题——学术型图书馆馆员如何启动数字人文项目,图书馆论坛,2017,20-25
1,1,"张舵,吴跃伟",国外图书馆支持数字人文的实践及启示,图书馆杂志,2014,47-52
2,2,"刘炜,谢蓉,张磊,等",面向人文研究的国家数据基础设施建设,中国图书馆学报,2016,29-39
3,3,"冯晴,陈惠兰",国外图书馆参与数字人文研究述评,图书馆杂志,2016,14-19
4,4,"高胜寒,赵宇翔,朱庆华",国内外数字人文领域研究进展分析,图书馆杂志,2016,9-18


In [568]:
# [1]肖鹏,彭嗣禹,王蕾. 基本原则与关键问题——学术型图书馆馆员如何启动数字人文项目[J]. 图书馆论坛, 2017, 卷缺失(期缺失): 20-25.
pd.Series('[',index=refs.index).str.cat(refs['index'].astype('str')).str.cat(refs.作者, sep='] ').str.cat(refs.标题,sep='. ').str.cat(refs.期刊,sep='[J]. ').str.cat(refs.年,sep=', ').str.cat(refs.页码,sep=': ').to_list()

['[0] 肖鹏,彭嗣禹,王蕾.  基本原则与关键问题——学术型图书馆馆员如何启动数字人文项目[J].  图书馆论坛,  2017:  20-25',
 '[1] 张舵,吴跃伟.  国外图书馆支持数字人文的实践及启示[J].  图书馆杂志,  2014:  47-52',
 '[2] 刘炜,谢蓉,张磊,等.  面向人文研究的国家数据基础设施建设[J].  中国图书馆学报,  2016:  29-39',
 '[3] 冯晴,陈惠兰.  国外图书馆参与数字人文研究述评[J].  图书馆杂志,  2016:  14-19',
 '[4] 高胜寒,赵宇翔,朱庆华.  国内外数字人文领域研究进展分析[J].  图书馆杂志,  2016:  9-18',
 '[5] 曾蕾,王晓光,范炜.  图档博领域的智慧数据及其在数字人文研究中的角色[J].  中国图书馆学报,  2018:  17-34',
 '[6] 赵生辉,朱学芳.  我国高校数字人文中心建设初探[J].  图书情报工作,  2014:  64-69',
 '[7] 熊莉君,张福阳,张灿.  图书馆在数字人文领域的传播功能与服务研究[J].  图书馆,  2016:  88-93',
 '[8] 邓要然,李少贞.  美国高校数字人文中心调查[J].  图书馆论坛,  2016:  26-34',
 '[9] 朱本军,聂华.  跨界与融合:全球视野下的数字人文——首届北京大学“数字人文论坛”会议综述[J].  大学图书馆学报,  2016:  16-21',
 '[10] 左娜,张卫东.  数字人文视角下的档案学研究[J].  图书与情报,  2019:  94-100',
 '[11] 朱娜.  数字人文的兴起及图书馆的角色[J].  图书馆,  2016:  17-22',
 '[12] 唐乐.  耶鲁大学图书馆的数字人文服务实践[J].  图书馆论坛,  2018:  10-18',
 '[13] 朱本军,聂华.  数字人文:图书馆实践的新方向[J].  大学图书馆学报,  2017:  23-29',
 '[14] 曾小莹.  数字人文背景下的图书馆:作用与服务[J].  图书与情报,  2014:  111-113',
 '[15] 柯平,宫平.  数字人文研究演化路径与热点领域分析[J].  中国图书馆学报,

In [ ]:
# 一些问题: 编号应该从1开始;卷期缺失需加进去;结尾应该再加一个'.';

### 2. 练习
#### 【练习一】 现有一份关于字符串的数据集，请解决以下问题：
#### （a）现对字符串编码存储人员信息（在编号后添加ID列），使用如下格式：“×××（名字）：×国人，性别×，生于×年×月×日”
#### （b）将（a）中的人员生日信息部分修改为用中文表示（如一九七四年十月二十三日），其余返回格式不变。
#### （c）将（b）中的ID列结果拆分为原列表相应的5列，并使用equals检验是否一致。

In [59]:
pd.read_csv('data/String_data_one.csv',index_col='人员编号').head()

,姓名,国籍,性别,出生年,出生月,出生日
人员编号,,,,,,
1,aesfd,2,男,1942,8,10
2,fasefa,5,女,1985,10,4
3,aeagd,4,女,1946,10,15
4,aef,4,男,1999,5,13
5,eaf,1,女,2010,6,24


In [575]:
df1=pd.read_csv('data/String_data_one.csv',index_col='人员编号')#.reset_index()
df1.head()

,姓名,国籍,性别,出生年,出生月,出生日
人员编号,,,,,,
1,aesfd,2,男,1942,8,10
2,fasefa,5,女,1985,10,4
3,aeagd,4,女,1946,10,15
4,aef,4,男,1999,5,13
5,eaf,1,女,2010,6,24


In [574]:
#for col in df1.columns[1:2]:
#    display(df1[col].str.cat(df1.人员编号.astype('str')))

In [586]:
df1['ID']=df1.姓名.str.cat(pd.Series('(名字)',index=df1.index)).str.cat(df1.国籍.astype('str'),sep=':').str.cat(df1.性别,sep='国人,性别').str.cat(df1.出生年.astype('str'),sep=',生于').str.cat(df1.出生月.astype('str'),sep='年').str.cat(df1.出生日.astype('str'),sep='月').str.cat(pd.Series('日',index=df1.index))

In [589]:
df1['ID'].head()

人员编号
1     aesfd(名字):2国人,性别男,生于1942年8月10日
2    fasefa(名字):5国人,性别女,生于1985年10月4日
3    aeagd(名字):4国人,性别女,生于1946年10月15日
4       aef(名字):4国人,性别男,生于1999年5月13日
5       eaf(名字):1国人,性别女,生于2010年6月24日
Name: ID, dtype: object

In [616]:
df1.出生年.apply(lambda x:list(str(x))).str[0]#.replace('','一')

人员编号
1       1
2       1
3       1
4       1
5       2
       ..
1996    1
1997    1
1998    2
1999    2
2000    1
Name: 出生年, Length: 2000, dtype: object

In [617]:
num2c={'1':'一','2':'二','3':'三','4':'四','5':'五','6':'六','7':'七','8':'八','9':'九','0':'零'}

In [631]:
df1.出生年.extract

Object `extract` not found.


In [641]:
df1.出生年=df1.出生年.astype('str').str.replace(r'(\d)(\d)(\d)(\d)',lambda x:x.group(1)).replace(num2c).str.cat(df1.出生年.astype('str').str.replace(r'(\d)(\d)(\d)(\d)',lambda x:x.group(2)).replace(num2c)).str.cat(df1.出生年.astype('str').str.replace(r'(\d)(\d)(\d)(\d)',lambda x:x.group(3)).replace(num2c)).str.cat(df1.出生年.astype('str').str.replace(r'(\d)(\d)(\d)(\d)',lambda x:x.group(4)).replace(num2c))
df1.出生年

人员编号
1       一九四二
2       一九八五
3       一九四六
4       一九九九
5       二零一零
        ... 
1996    一九八四
1997    一九四三
1998    二零一八
1999    二零零五
2000    一九六二
Name: 出生年, Length: 2000, dtype: object

In [734]:
df11=df1['ID'].str.split(r'\(名字\)\:',expand=True)
df11=df11.iloc[:,0].to_frame().join(df11[1].str.split('国人,性别',expand=True),rsuffix='_')
df11=df11.iloc[:,:2].join(df11[1].str.split(',生于',expand=True),rsuffix='_')
df11=df11.iloc[:,:3].join(df11[1].str.split('年',expand=True),rsuffix='_')
df11=df11.iloc[:,:4].join(df11[1].str.split('月',expand=True),rsuffix='_')
df11=df11.iloc[:,:5].join(df11[1].str.split('日',expand=True),rsuffix='_')
df11=df11.iloc[:,:6]

df11.columns=df1.columns[:6]
df11

,姓名,国籍,性别,出生年,出生月,出生日
人员编号,,,,,,
1,aesfd,2,男,1942,8,10
2,fasefa,5,女,1985,10,4
3,aeagd,4,女,1946,10,15
4,aef,4,男,1999,5,13
5,eaf,1,女,2010,6,24
...,...,...,...,...,...,...
1996,sdf,5,男,1984,4,17
1997,hx,1,男,1943,7,16
1998,drg,5,女,2018,4,6


In [740]:
df11.equals(pd.read_csv('data/String_data_one.csv',index_col='人员编号').astype('str'))

True

#### 【练习二】 现有一份半虚拟的数据集，第一列包含了新型冠状病毒的一些新闻标题，请解决以下问题：
#### （a）选出所有关于北京市和上海市新闻标题的所在行。

In [60]:
pd.read_csv('data/String_data_two.csv').head()

,col1,col2,col3
0,鄂尔多斯市第2例确诊患者治愈出院,19,363.6923
1,云南新增2例，累计124例,-67,-152.281
2,武汉协和医院14名感染医护出院,-86,325.6221
3,山东新增9例，累计307例,-74,-204.9313
4,上海开学日期延至3月,-95,4.05


In [741]:
df2=pd.read_csv('data/String_data_two.csv')

In [744]:
# 北京
df2[df2.col1.str.contains('北京')]
#上海
df2[df2.col1.str.contains('上海')]

,col1,col2,col3
4,上海开学日期延至3月,-95,4.05
6,上海新增10例，累计243例,2,-73.7105
36,上海新增14例累计233例,-55,-83
40,上海新增14例累计233例,-88,-99
53,上海7个月大婴儿确诊,-71,-391.2325
80,正直播！上海疫情防控发布会,-52,-211.4114
114,上海新增5例累计208例,100,327.2421
145,上海7个月婴儿被感染,-69,140.9409
178,上海新增11例累计确诊193例,69,150.661
204,上海新增5例累计确诊182例,20,-25.9202


#### （b）求col2的均值。

In [763]:
lst=[]
for v in df2.col2.values:
    try:
        lst.append(int(v))
    except:
        lst.append(0)
lst
np.array(lst).mean()

-1.016

#### （c）求col3的均值。

In [768]:
lst=[]
for v in df2['col3  '].values:
    try:
        lst.append(float(v))
    except:
        lst.append(0)
lst
np.array(lst).mean()

-1.1849898000000014

In [767]:
df2.columns

Index(['col1', 'col2', 'col3  '], dtype='object')